In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 1. 데이터 불러오기

df = pd.read_csv('../processed_data/processed_features.csv')

print(f"데이터 크기: {df.shape}")
print(f"컬럼: {list(df.columns)}")
print("\n첫 5행:")
print(df.head())


데이터 크기: (2156, 36)
컬럼: ['file', 'ch1_rms', 'ch1_peak', 'ch1_std', 'ch1_kurtosis', 'ch2_rms', 'ch2_peak', 'ch2_std', 'ch2_kurtosis', 'ch3_rms', 'ch3_peak', 'ch3_std', 'ch3_kurtosis', 'ch4_rms', 'ch4_peak', 'ch4_std', 'ch4_kurtosis', 'ch5_rms', 'ch5_peak', 'ch5_std', 'ch5_kurtosis', 'ch6_rms', 'ch6_peak', 'ch6_std', 'ch6_kurtosis', 'ch7_rms', 'ch7_peak', 'ch7_std', 'ch7_kurtosis', 'ch8_rms', 'ch8_peak', 'ch8_std', 'ch8_kurtosis', 'timestamp', 'time_to_failure', 'days_to_failure']

첫 5행:
                  file   ch1_rms  ch1_peak   ch1_std  ch1_kurtosis   ch2_rms  \
0  2003.10.22.12.06.24  0.124614     0.720  0.081122      1.069717  0.117493   
1  2003.10.22.12.09.13  0.123811     0.654  0.079515      1.162128  0.116833   
2  2003.10.22.12.14.13  0.125246     0.623  0.080217      0.986819  0.118384   
3  2003.10.22.12.19.13  0.125197     0.598  0.080825      1.034839  0.119005   
4  2003.10.22.12.24.13  0.125618     0.623  0.082034      1.110728  0.119688   

   ch2_peak   ch2_std  ch2_ku

In [4]:
# 2. 라벨 생성

# 고장 D-6일 이내면 고장(1), 아니면 정상(0)
threshold_days = 6
df['label'] = (df['days_to_failure'] <= threshold_days).astype(int)

# 라벨 분포 확인
label_counts = df['label'].value_counts()
label_ratios = df['label'].value_counts(normalize=True)

print(f"라벨 분포:")
print(f"  정상 (0): {label_counts[0]:,}개 ({label_ratios[0]*100:.1f}%)")
print(f"  고장 (1): {label_counts[1]:,}개 ({label_ratios[1]*100:.1f}%)")
print(f"  임계값: D-{threshold_days}일")

라벨 분포:
  정상 (0): 1,507개 (69.9%)
  고장 (1): 649개 (30.1%)
  임계값: D-6일


In [5]:
# 3. 특성 컬럼 선택 및 데이터 분할 (계층적)

# 특성 컬럼
feature_cols = [col for col in df.columns if col.startswith('ch')]
print(f"특성 컬럼 수: {len(feature_cols)}개")
print(f"특성 컬럼: {feature_cols[:5]} ...")

# 계층적 분할 (클래스별 70:15:15)
def split_data(data, train_ratio=0.7, val_ratio=0.15):
    n = len(data)
    train_end = int(n * train_ratio)
    val_end = train_end + int(n * val_ratio)
    
    train = data.iloc[:train_end]
    val = data.iloc[train_end:val_end]
    test = data.iloc[val_end:]
    
    return train, val, test

# 정상/고장 데이터 분리
normal_data = df[df['label']==0].copy()
failure_data = df[df['label']==1].copy()

print(f"\n데이터 분리:")
print(f"  정상 데이터: {len(normal_data):,}개")
print(f"  고장 데이터: {len(failure_data):,}개")

# 각각 분할
normal_train, normal_val, normal_test = split_data(normal_data)
failure_train, failure_val, failure_test = split_data(failure_data)

# 합치기 및 섞기
train_data = pd.concat([normal_train, failure_train], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
val_data = pd.concat([normal_val, failure_val], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
test_data = pd.concat([normal_test, failure_test], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n분할 결과:")
print(f"  Train: {train_data.shape[0]:,}개 (고장 {(train_data['label']==1).sum():,}개, {(train_data['label']==1).sum()/len(train_data)*100:.1f}%)")
print(f"  Val:   {val_data.shape[0]:,}개 (고장 {(val_data['label']==1).sum():,}개, {(val_data['label']==1).sum()/len(val_data)*100:.1f}%)")
print(f"  Test:  {test_data.shape[0]:,}개 (고장 {(test_data['label']==1).sum():,}개, {(test_data['label']==1).sum()/len(test_data)*100:.1f}%)")

특성 컬럼 수: 32개
특성 컬럼: ['ch1_rms', 'ch1_peak', 'ch1_std', 'ch1_kurtosis', 'ch2_rms'] ...

데이터 분리:
  정상 데이터: 1,507개
  고장 데이터: 649개

분할 결과:
  Train: 1,508개 (고장 454개, 30.1%)
  Val:   323개 (고장 97개, 30.0%)
  Test:  325개 (고장 98개, 30.2%)


In [6]:
# 4. 시계열 윈도우 생성

window_size = 10  # 과거 10개 시점으로 예측

def create_sequences(data, feature_cols, window_size):
    # 시계열 윈도우 생성
    X = []
    y = []
    
    # timestamp로 정렬
    data = data.sort_values('timestamp').reset_index(drop=True)
    
    # 슬라이딩 윈도우
    for i in range(len(data) - window_size):
        window = data.iloc[i:i+window_size][feature_cols].values
        label = data.iloc[i+window_size]['label']
        
        X.append(window)
        y.append(label)
    
    return np.array(X), np.array(y)

# 윈도우 생성
print(f"윈도우 크기: {window_size}")
print("윈도우 생성 중...")

X_train, y_train = create_sequences(train_data, feature_cols, window_size)
X_val, y_val = create_sequences(val_data, feature_cols, window_size)
X_test, y_test = create_sequences(test_data, feature_cols, window_size)

print(f"\n윈도우 생성 완료:")
print(f"  X_train: {X_train.shape} (samples, timesteps, features)")
print(f"  X_val:   {X_val.shape}")
print(f"  X_test:  {X_test.shape}")
print(f"  y_train: {y_train.shape} (고장 비율: {y_train.sum()/len(y_train)*100:.1f}%)")
print(f"  y_val:   {y_val.shape} (고장 비율: {y_val.sum()/len(y_val)*100:.1f}%)")
print(f"  y_test:  {y_test.shape} (고장 비율: {y_test.sum()/len(y_test)*100:.1f}%)")

윈도우 크기: 10
윈도우 생성 중...

윈도우 생성 완료:
  X_train: (1498, 10, 32) (samples, timesteps, features)
  X_val:   (313, 10, 32)
  X_test:  (315, 10, 32)
  y_train: (1498,) (고장 비율: 30.3%)
  y_val:   (313,) (고장 비율: 31.0%)
  y_test:  (315,) (고장 비율: 31.1%)


In [7]:
# 5. 데이터 정규화

print("데이터 정규화 중...")

# 3D → 2D 변환
X_train_2d = X_train.reshape(-1, X_train.shape[-1])
X_val_2d = X_val.reshape(-1, X_val.shape[-1])
X_test_2d = X_test.reshape(-1, X_test.shape[-1])

# 스케일러 생성 및 학습
scaler = StandardScaler()
scaler.fit(X_train_2d)

# 정규화 적용
X_train_scaled_2d = scaler.transform(X_train_2d)
X_val_scaled_2d = scaler.transform(X_val_2d)
X_test_scaled_2d = scaler.transform(X_test_2d)

# 3D로 복원
X_train_scaled = X_train_scaled_2d.reshape(X_train.shape)
X_val_scaled = X_val_scaled_2d.reshape(X_val.shape)
X_test_scaled = X_test_scaled_2d.reshape(X_test.shape)

print(f"정규화 완료:")
print(f"  정규화 후 평균: {X_train_scaled_2d.mean(axis=0)[:3]} ... (거의 0)")
print(f"  정규화 후 표준편차: {X_train_scaled_2d.std(axis=0)[:3]} ... (거의 1)")
print(f"  스케일러 특성 수: {scaler.n_features_in_}개")

데이터 정규화 중...
정규화 완료:
  정규화 후 평균: [-5.13487561e-14  3.19595855e-14  1.48720214e-13] ... (거의 0)
  정규화 후 표준편차: [1. 1. 1.] ... (거의 1)
  스케일러 특성 수: 32개


In [8]:
# 6. 전처리된 데이터 저장

# 저장 경로
processed_data_dir = '../processed_data'
models_dir = '../models'

# 폴더 생성
os.makedirs(processed_data_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

print("전처리된 데이터 저장 중...")

# 3D 구조 보존한 PKL 저장 (딥러닝 모델용)
joblib.dump(X_train_scaled, f'{processed_data_dir}/X_train_scaled.pkl')
joblib.dump(X_val_scaled, f'{processed_data_dir}/X_val_scaled.pkl')
joblib.dump(X_test_scaled, f'{processed_data_dir}/X_test_scaled.pkl')

joblib.dump(y_train, f'{processed_data_dir}/y_train.pkl')
joblib.dump(y_val, f'{processed_data_dir}/y_val.pkl')
joblib.dump(y_test, f'{processed_data_dir}/y_test.pkl')

# 스케일러 저장 (모든 모델에서 공유)
joblib.dump(scaler, f'{models_dir}/scaler.pkl')

print(f"\n저장 완료:")
print(f"  - X_train_scaled.pkl: {X_train_scaled.shape}")
print(f"  - X_val_scaled.pkl: {X_val_scaled.shape}")  
print(f"  - X_test_scaled.pkl: {X_test_scaled.shape}")
print(f"  - y_train.pkl: {y_train.shape}")
print(f"  - y_val.pkl: {y_val.shape}")
print(f"  - y_test.pkl: {y_test.shape}")
print(f"  - scaler.pkl: 정규화 스케일러")

전처리된 데이터 저장 중...

저장 완료:
특성 데이터:
  - ../processed_data/X_train_scaled.pkl ((1498, 10, 32))
  - ../processed_data/X_val_scaled.pkl ((313, 10, 32))
  - ../processed_data/X_test_scaled.pkl ((315, 10, 32))
라벨 데이터:
  - ../processed_data/y_train.pkl ((1498,))
  - ../processed_data/y_val.pkl ((313,))
  - ../processed_data/y_test.pkl ((315,))
공통 스케일러:
  - ../models/scaler.pkl (모든 모델에서 사용)

다음 단계:
  - 03_lstm.ipynb에서 이 데이터들을 로드해서 LSTM 모델 학습
  - 04_cnn.ipynb, 05_transformer.ipynb 등에서도 동일한 데이터 사용
  - 모든 모델이 동일한 전처리 기준으로 공정한 비교 가능
